### Importieren benötigter Bibliotheken

In [ ]:
from __future__ import print_function, division
from sympy import symbols, simplify, trigsimp
from sympy.physics.mechanics import dynamicsymbols, ReferenceFrame, Point
from sympy.physics.mechanics import inertia, RigidBody, KanesMethod
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)

# 1. Kinematik

### Koordinatensysteme / Bezugssysteme

In [ ]:
# Initialisieren der benötigten Koordinatensysteme
inertial_frame = ReferenceFrame('I')
pendulum1_frame = ReferenceFrame('P_1')

### Orientierung und Zwangsbedingungen an den Koordinatensystemen

In [ ]:
theta1 = dynamicsymbols('theta1')

In [ ]:
# Zwangsbedingung des Bezugssystems P1 zum Inertialsystem:
# setze fest, dass pendulum1 lediglich um die z-Achse des Inertalsystems rotieren kann, um den Winkel theta1
pendulum1_frame.orient(inertial_frame, 'Axis', (theta1, inertial_frame.z))

In [ ]:
# Daraus ergibt sich dann, dass die die Position von pendulum1 mit der folgenden Rotationsmatrix berechnet werden kann:
pendulum1_frame.dcm(inertial_frame)

<p> Multipliziert man diese Rotationsmatrix mit einem Vektor, lässt sich damit der Vektor um die entsprechenden Winkel zum Inertialsystem drehen</p>

### Massepunkte und Gelenkpunkte

In [ ]:
# Zunächst setzen wir einen fixen Ankerpunkt im Inertialsystem
ankle = Point('A')

In [ ]:
# setzen unseren zweiten Punkt, der später pendulum1 und pendulum2 verbinden soll
p1 = Point('P_1')

In [ ]:
# definieren nun eine Länge für unser erstes Pendel
pendulum1_length = symbols('l_1')

In [ ]:
# Die Position des Punktes P lässt sich jetzt über die Länge l1 * der Richtung y des Bezugssystems P beschreiben
p1.set_pos(ankle, pendulum1_length * pendulum1_frame.y)

# Durch die Multiplikation mit der Rotationsmatrix zwischen dem Inertialsystems und Pendulum1 kann der Punkt P
# im Bezug zum Inertialsystem beschrieben werden
p1.pos_from(ankle).express(inertial_frame).simplify()

<p>Da wir nicht mit masselosen Pendeln rechnen wollen, müssen wir außerdem die Massenschwerpunkte unserer Pendel angeben</p>

In [ ]:
# wir machen dies analog zur Vorgehensweise der Gelenkpunkte
pendulum1_mass_length = symbols('l_m1')
pendulum1_mass_center = Point('P_m1')
pendulum1_mass_center.set_pos(ankle, pendulum1_mass_length * pendulum1_frame.y)

### Aufstellen der Differentialgleichungen

In [ ]:
# Wir benötigen die Winkelgeschwindigkeiten
omega1 = dynamicsymbols('omega1')

# Diese sind genauso groß, wie die Ableitung des Winkels Theta1 nach der Zeit --> 𝜔 = 𝜃˙
# Diese Formulierung müssen wir tätigen, da wir unsere Bewegungsgleichung nach Kanes-Methode aufstellen.
# Zudem benötigen alle numerischen Integratoren eine DGL 1.Ordnung
kinematical_differential_equations = [omega1 - theta1.diff()]
kinematical_differential_equations

In [ ]:
# Die erste Winkelgeschwindigkeit rotiert um die Z-Achse unseres Inertialsystems
pendulum1_frame.set_ang_vel(inertial_frame, omega1*inertial_frame.z)
pendulum1_frame.ang_vel_in(inertial_frame)

In [ ]:
# Der Ankerpunkt am Inertialsystem wird raumfest mit der Geschwindigkeit 0 angenommen 
ankle.set_vel(inertial_frame, 0)

# 2. Massenträgheiten

### Massenträgheiten

In [ ]:
pendulum1_mass = symbols('m_1')

In [ ]:
pendulum1_inertia = symbols('I_p1')

In [ ]:
help(inertia)

In [ ]:
# Das Dyadsiche Produkt ermöglicht uns das speichern der Trägheit als Tensor in Abhängigkeit zum Bezugssystem
pendulum1_inertia_dyadic = inertia(pendulum1_frame, 0, 0, pendulum1_inertia)
pendulum1_inertia_dyadic

In [ ]:
pendulum1_inertia_dyadic.to_matrix(pendulum1_frame)

In [ ]:
pendulum1_central_inertia = (pendulum1_inertia_dyadic, pendulum1_mass_center)

In [ ]:
# Eine komplette Beschreibung des Rigid Bodys besteht aus: Massenschwerpunkt, Bezugssystem, Masse und Trägheit
pendulum1 = RigidBody('Pendulum 1', pendulum1_mass_center, pendulum1_frame,
                      pendulum1_mass, pendulum1_central_inertia)

# 3. Kinetik (extern aufgeprägte Kräfte)

In [ ]:
g = symbols('g')

In [ ]:
pendulum1_grav_force_vector = -pendulum1_mass * g * inertial_frame.y
pendulum1_grav_force_vector

In [ ]:
# Die Gravitationskraft legen wir immer mit -y Richtung unseres Inertialsystems an
pendulum1_grav_force = (pendulum1_mass_center, pendulum1_grav_force_vector)

In [ ]:
# Zusätzlich könnten wir an unserem Rotations-FHG der Gelenke auch ein Moment aufprägen
ankle_torque, p1_torque = dynamicsymbols('T_a, T_p1')
pendulum1_torque_vector = ankle_torque * inertial_frame.z - p1_torque * inertial_frame.z
pendulum1_torque = (pendulum1_frame, pendulum1_torque_vector)
pendulum1_torque_vector

# 4. Aufstellen der Bewegungsgleichungen

In [ ]:
angles = [theta1]
speeds = [omega1]
# Wir hatten dies bereits im Kapitel 1 - Kinematik definiert
kinematical_differential_equations

In [ ]:
# Zusammentragen aller von außen aufgeprägten Kräfte
loads = [pendulum1_grav_force,
         pendulum1_torque]
loads

In [ ]:
# Wir erinnern uns an die angelegten Starrkörper
bodies = [pendulum1]
bodies

<p>Die Formulierung der Bewegungsgleichungen nach Kane sieht der bekannten Form nach Newton Euler sehr ähnlich:</p>
<p>Kanes Formulierung: $F_r + F_r^* = 0$</p>
<p>Newton Euler Formulierung: ${F} = {m}{a} \hspace{1cm} {T} = {I}{\alpha}$</p>

In [ ]:
kane = KanesMethod(inertial_frame, angles, speeds, kinematical_differential_equations)
fr, frstar = kane.kanes_equations(bodies, loads)

In [ ]:
trigsimp(fr + frstar)

In [ ]:
mass_matrix = trigsimp(kane.mass_matrix_full)
mass_matrix

In [ ]:
forcing_vector = trigsimp(kane.forcing_full)
forcing_vector

### Damit haben wir alle Dinge beisammen:
<p>- Bezugssysteme</p>
<p>- Gelenkpunkte + Zwangsbedingungen</p>
<p>- Massepunkte</p>
<p>- Massenträgheit</p>
<p>- Kinetik (externe Kräfte)</p>
<p>- Formulierung der Bewegungsgleichungen</p>
<p>- Umformen für die Simulation in Massenabhängikeitsmatrix und Kraftabhängigkeitsmatrix als DGL 1.Ordnung $M(x,t)\dot x = f(x,t)$ (x=statevector)</p>

# 5. Lösen der Bewegungsgleichung

In [ ]:
from numpy import deg2rad, rad2deg, array, zeros, linspace
from scipy.integrate import odeint
from pydy.codegen.ode_function_generators import generate_ode_function
from matplotlib.pyplot import plot, legend, xlabel, ylabel, rcParams
from sympy.physics.vector import init_vprinting, vlatex
%matplotlib inline
rcParams['figure.figsize'] = (14.0, 6.0)

In [ ]:
constants = [pendulum1_length,
             pendulum1_mass_length,
             pendulum1_mass,
             pendulum1_inertia,
             g]
constants

In [ ]:
specified = [ankle_torque, p1_torque]

In [ ]:
# wir lassen uns die Gleichung nach x' umstellen
right_hand_side = generate_ode_function(forcing_vector, angles,
                                        speeds, constants,
                                        mass_matrix=mass_matrix,
                                        specifieds=specified)

In [ ]:
help(right_hand_side)

In [ ]:
# Festlegen der Initialbedingungen für die numerische Lösung der DGL

# Zu Beginn setzen wir beide Geschwindigkeiten auf 0
# und die beiden Winkel versetzen wir nach belieben
x0 = zeros(2)
x0[0] = deg2rad(120)

# Wir setzen nun realistische Werte für unsere Konstanten an
numerical_constants = array([0.8,  # pendulum1_length [m]
                             0.4,  # pendulum1_com_length [m]
                             4.0,  # pendulum1_mass [kg]
                             0.64,  # pendulum1_inertia [kg*m^2]
                             9.81],  # acceleration due to gravity [m/s^2]
                            )

# Wir setzen auch die Gelenkmomente initial auf 0
numerical_specified = array([0, 0])

<p>Mit den bekannten Anfangsbedingungen wollen wir über unseren gewünschten Beobachtungszeitraum numerisch integrieren.</p>
<p>Dazu geben wir dem Solver (z. B. Runge-Kutta) unsere Anfangsbedingungen, die Gleichung für x' und den Zeitraum sowie die Schrittweite t.</p>

In [ ]:
frames_per_sec = 30
final_time = 8

t = linspace(0, final_time, final_time * frames_per_sec)

y = odeint(right_hand_side, x0, t, args=(numerical_specified, numerical_constants))

In [ ]:
y.shape

# 6. Visualisierung

In [ ]:
# Darstellung der zwei Winkel über die Simulationsdauer
plot(t, rad2deg(y[:, :1]))
xlabel('Zeit [s]')
ylabel('Winkel [deg]')
legend(["${}$".format(vlatex(c)) for c in angles])

In [ ]:
# Darstellung der beiden Geschwindigkeiten über die Simulationsdauer
plot(t, rad2deg(y[:, 1:]), color='r') # siehe y.shape
xlabel('Zeit [s]')
ylabel('Winkelgeschw. [deg/s]')
legend(["${}$".format(vlatex(s)) for s in speeds])

In [ ]:
from pydy.viz.shapes import Cylinder, Sphere
import pydy.viz
from pydy.viz.visualization_frame import VisualizationFrame
from pydy.viz.scene import Scene

# Für die Punkte legen wir schwarze Kugeln an
ankle_shape = Sphere(color='black', radius=0.1)
p1_shape = Sphere(color='black', radius=0.1)

# setzen die erzeugten Geometrien an die Punkte
ankle_viz_frame = VisualizationFrame(inertial_frame, ankle, ankle_shape)
p1_viz_frame = VisualizationFrame(inertial_frame, p1, p1_shape)

# umformen der Konstanten in für das Programm leserliches Format
constants_dict = dict(zip(constants, numerical_constants))

# Setzen der Geometrie für die Starrkörper
pendulum1_center = Point('p1_c')
pendulum1_center.set_pos(ankle, pendulum1_length / 2 * pendulum1_frame.y)

pendulum1_shape = Cylinder(radius=0.08, length=constants_dict[pendulum1_length], color='blue')
pendulum1_viz_frame = VisualizationFrame('Pendulum 1', pendulum1_frame, pendulum1_center, pendulum1_shape)

# Erstellen der Szene mit dem Inertialsystem/Ankerpunkt in der Mitte
scene = Scene(inertial_frame, ankle)

# Laden der Starrkörper in die Szene
scene.visualization_frames = [ankle_viz_frame,
                              p1_viz_frame,
                              pendulum1_viz_frame]

# Darstellung und einfügen der Ergebnisse
scene.states_symbols = angles + speeds
scene.constants = constants_dict
scene.states_trajectories = y

# öffnen der Simulation
scene.display()